In [110]:
%pip -q install numpy ipytest pytest

import ipytest
import pytest
ipytest.autoconfig()
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [111]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix
    
    def T(self):
        result = []
        for j in range(len(self.matrix[0])):
            row = []
            for i in range(len(self.matrix)):
                row.append(self.matrix[i][j])
            result.append(row)
        
        return Matrix(result)

    def __add__(self, other):
        if isinstance(other, (int, float)):
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] + other)
                result.append(row)
            return Matrix(result)
        elif isinstance(other, Matrix):
            if len(self.matrix) != len(other.matrix) or len(self.matrix[0]) != len(other.matrix[0]):
                raise ValueError("Matrices must have the same dimensions for addition")
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] + other.matrix[i][j])
                result.append(row)
            return Matrix(result)
        else:
            raise TypeError("Unsupported operand type for +: 'Matrix' and '{}'".format(type(other).__name__))
            
    def __sub__(self, other):
        if isinstance(other, (int, float)):
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] - other)
                result.append(row)
            return Matrix(result)
        elif isinstance(other, Matrix):
            if len(self.matrix) != len(other.matrix) or len(self.matrix[0]) != len(other.matrix[0]):
                raise ValueError("Matrices must have the same dimensions for subtraction")
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] - other.matrix[i][j])
                result.append(row)
            return Matrix(result)
        else:
            raise TypeError("Unsupported operand type for -: 'Matrix' and '{}'".format(type(other).__name__))
            
    def __mul__(self, other):
        if isinstance(other, Matrix):
            if len(self.matrix[0]) != len(other.matrix):
                raise ValueError("The number of columns in the first matrix must be equal to the number of rows in the second matrix for multiplication")
            
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(other.matrix[0])):
                    sum = 0
                    for k in range(len(other.matrix)):
                        sum += self.matrix[i][k] * other.matrix[k][j]
                    row.append(sum)
                result.append(row)
            
            return Matrix(result)
        
        elif isinstance(other, (int, float)):
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] * other)
                result.append(row)
            
            return Matrix(result)
        
        else:
            raise ValueError("Multiplication is not defined for the specified type")
            
    def __truediv__(self, other):
        if isinstance(other, (int, float)):
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] / other)
                result.append(row)
            return Matrix(result)
        elif isinstance(other, Matrix):
            if len(self.matrix) != len(other.matrix) or len(self.matrix[0]) != len(other.matrix[0]):
                raise ValueError("Matrices must have the same dimensions for division")
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(self.matrix[0])):
                    row.append(self.matrix[i][j] / other.matrix[i][j])
                result.append(row)
            return Matrix(result)
        else:
            raise TypeError("Unsupported operand type for /: 'Matrix' and '{}'".format(type(other).__name__))

    def __matmul__(self, other):
        if isinstance(other, Matrix):
            result = []
            for i in range(len(self.matrix)):
                row = []
                for j in range(len(other.matrix[0])):
                    sum = 0
                    for k in range(len(other.matrix)):
                        sum += self.matrix[i][k] * other.matrix[k][j]
                    row.append(sum)
                result.append(row)
            return Matrix(result)
        else:
            raise TypeError("Unsupported operand type for @: 'Matrix' and '{}'".format(type(other).__name__))
    

In [112]:
%%ipytest

_matrix1 = [[1, 2, 3], 
           [4, 5, 6],
           [10, 11, 12]]

_matrix2 = [[7, 8, 9], 
           [10, 11, 12],
           [3, 5, 7]]

matrix1 = Matrix(_matrix1)
matrix2 = Matrix(_matrix2)

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_add_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 + matrix2).matrix) == (np.array(matrix1.matrix) + np.array(matrix2.matrix))).all()
    ), 'add section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13]])
def test_matrix_mul_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 * number).matrix) == np.array(matrix1.matrix) * number).all()
    ), 'mul section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])    
def test_matrix_div_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 / matrix2).matrix) == np.array(matrix1.matrix) / np.array(matrix2.matrix)).all()
    ), 'div section with matrices is not working right'

@pytest.mark.parametrize("matrix1", [matrix1])    
def test_matrix_transpose(matrix1: Matrix):
    assert (
        (np.array(matrix1.T().matrix) == np.array(matrix1.matrix).T).all()
    ), 'transpose section is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_sub_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 - matrix2).matrix) == np.array(matrix1.matrix) - np.array(matrix2.matrix)).all()
    ), 'sub section with matrices is not working right'
    
# -----------------------------------------

@pytest.mark.parametrize("matrix1,number", [[matrix1, 2]])    
def test_matrix_div_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 / number).matrix) == np.array(matrix1.matrix) / number).all()
    ), 'div section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2.T()]])    
def test_matrix_matmul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert(
        (np.array((matrix1 @ matrix2).matrix) == np.array(matrix1.matrix) @ np.array(matrix2.matrix)).all()
    ), 'matmul section is not working right, and don\'t forget about T() if needed'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13.2]])
def test_matrix_add_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 + number).matrix) == np.array(matrix1.matrix) + number).all()
    ), 'add section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 12.2]])    
def test_matrix_sub_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 - number).matrix) == np.array(matrix1.matrix) - number).all()
    ), 'sub section with matrix and number is not working right'

_matrix1 = [[1, 2, 3], 
           [4, 5, 6],
           [4, 5, 6]]

_matrix2 = [[7, 8, 9], 
           [10, 11, 12],
           [3, 5, 7]]

matrix1 = Matrix(_matrix1)
matrix2 = Matrix(_matrix2)

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]]) 
def test_matrix_mul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 * matrix2).matrix) == np.array(matrix1.matrix) * np.array(matrix2.matrix)).all()
    ), 'mul section with matrices is not working right'
    

.........F                                                                                   [100%]
============================================ FAILURES =============================================
____________________________ test_matrix_mul_matrix[matrix10-matrix20] ____________________________

matrix1 = <__main__.Matrix object at 0x0000027CC9CA6350>
matrix2 = <__main__.Matrix object at 0x0000027CC9CA7010>

    @pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
    def test_matrix_mul_matrix(matrix1: Matrix, matrix2: Matrix):
>       assert (
            (np.array((matrix1 * matrix2).matrix) == np.array(matrix1.matrix) * np.array(matrix2.matrix)).all()
        ), 'mul section with matrices is not working right'
E       AssertionError: mul section with matrices is not working right
E       assert False
E        +  where False = <built-in method all of numpy.ndarray object at 0x0000027CC9F467F0>()
E        +    where <built-in method all of numpy.ndarray object at 0x0